In [42]:
import sys
import importlib
from urllib.request import urlretrieve

# Install drake (and underactuated).
if 'google.colab' in sys.modules and importlib.util.find_spec('underactuated') is None:
    urlretrieve(f"http://underactuated.csail.mit.edu/scripts/setup/setup_underactuated_colab.py",
                "setup_underactuated_colab.py")
    from setup_underactuated_colab import setup_underactuated
    setup_underactuated(underactuated_sha='560c2adace05eb20ebd78377582015d5b2d3859a', drake_version='0.27.0', drake_build='release')

In [43]:
# Do general python imports

import os
import numpy as np
import matplotlib.pyplot as plt

# Drake Imports

from pydrake.all import PiecewisePolynomial

# Pieceswise Polynomial of a Time Varying Linear System

We are trying to create a `TimeVaryingAffineSystem` which requires `PiecewisePolynomial` as inputs. For the $A(t)$ time varying state matrix, we take a 2x2 matrix which signifies changing dynamics in this case. Similar example can be expanded to include real time-varying dynamics i.e TVLQR.

Here we consider the following time-steps for our system: t = [0.1, 0.2, 0.3]. For these time-steps, the time-varying dynamics can be given as:
$$
A(t[0]) = A(t=0.1) = A_1 = 
\begin{bmatrix}
0.1 & 0 \\
0 & 0.1
\end{bmatrix}
$$

$$
A(t[1]) = A(t=0.2) = A_2 =
\begin{bmatrix}
0.2 & 0 \\
0 & 0.2
\end{bmatrix}
$$

$$
A(t[2]) = A(t=0.3) = A_3 =
\begin{bmatrix}
0.3 & 0 \\
0 & 0.3
\end{bmatrix}
$$

for all of these, the matrix $B = [0, 1]^T$.

Using the above time-varying linear system to try to create a `TimeVaryingAffineSystem`. For this, we need to create a `PiecewisePolynomial` for the time-varying A and B matrices.

In [44]:
A1 = np.array([[0.1, 0], [0, 0.1]])
A2 = np.array([[0.2, 0], [0, 0.2]])
A3 = np.array([[0.3, 0], [0, 0.3]])

# A1 = np.asfortranarray(A1)
# A2 = np.asfortranarray(A2)
# A3 = np.asfortranarray(A3)

t = [0.1, 0.2, 0.3]

B1 = np.array([[0], [1]])
B2 = np.array([[0], [1]])
B3 = np.array([[0], [1]])

B = np.hstack([B1, B2, B3])

We now try to create a `PiecewisePolynomial` without success by initializing using the matrices in different ways:

In [45]:
PolyA = PiecewisePolynomial(np.array([A1, A2, A3]), t)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. pydrake.trajectories.PiecewisePolynomial()
    2. pydrake.trajectories.PiecewisePolynomial(arg0: numpy.ndarray[numpy.float64[m, n], flags.f_contiguous])
    3. pydrake.trajectories.PiecewisePolynomial(arg0: List[numpy.ndarray[object[m, n]]], arg1: List[float])
    4. pydrake.trajectories.PiecewisePolynomial(arg0: List[pydrake.polynomial.Polynomial], arg1: List[float])

Invoked with: array([[[0.1, 0. ],
        [0. , 0.1]],

       [[0.2, 0. ],
        [0. , 0.2]],

       [[0.3, 0. ],
        [0. , 0.3]]]), [0.1, 0.2, 0.3]

In [46]:
PolyA = PiecewisePolynomial(np.array([[A1], [A2], [A3]]), t)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. pydrake.trajectories.PiecewisePolynomial()
    2. pydrake.trajectories.PiecewisePolynomial(arg0: numpy.ndarray[numpy.float64[m, n], flags.f_contiguous])
    3. pydrake.trajectories.PiecewisePolynomial(arg0: List[numpy.ndarray[object[m, n]]], arg1: List[float])
    4. pydrake.trajectories.PiecewisePolynomial(arg0: List[pydrake.polynomial.Polynomial], arg1: List[float])

Invoked with: array([[[[0.1, 0. ],
         [0. , 0.1]]],


       [[[0.2, 0. ],
         [0. , 0.2]]],


       [[[0.3, 0. ],
         [0. , 0.3]]]]), [0.1, 0.2, 0.3]

Looking at the above type errors, we specifically tailer the initialization to the following acceptable type (Type 3):

`3. pydrake.trajectories.PiecewisePolynomial(arg0: List[numpy.ndarray[object[m, n]]], arg1: List[float])`

Try with passing a list of numpy arrays as the first argument and the list of time steps at the second argument

In [47]:
PolyA = PiecewisePolynomial([A1, A2, A3], t)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. pydrake.trajectories.PiecewisePolynomial()
    2. pydrake.trajectories.PiecewisePolynomial(arg0: numpy.ndarray[numpy.float64[m, n], flags.f_contiguous])
    3. pydrake.trajectories.PiecewisePolynomial(arg0: List[numpy.ndarray[object[m, n]]], arg1: List[float])
    4. pydrake.trajectories.PiecewisePolynomial(arg0: List[pydrake.polynomial.Polynomial], arg1: List[float])

Invoked with: [array([[0.1, 0. ],
       [0. , 0.1]]), array([[0.2, 0. ],
       [0. , 0.2]]), array([[0.3, 0. ],
       [0. , 0.3]])], [0.1, 0.2, 0.3]

However, making a `PiecewisePolynomial` with a time-varying 1D array (instead of 2D array) is easier as shown below:

In [48]:
PolyB = PiecewisePolynomial.FirstOrderHold(t, B)

Using similar method for 2D array fails:

In [49]:
PolyA = PiecewisePolynomial.FirstOrderHold(t, np.hstack([A1, A2, A3]))

SystemExit: Failure at common/trajectories/piecewise_polynomial.cc:1116 in FirstOrderHold(): condition 'samples.cols() == breaks.size()' failed.

/home/dfki.uni-bremen.de/svyas/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [56]:
A = np.array([A1, A2, A3]).T
PolyA = PiecewisePolynomial.FirstOrderHold(t, A)

TypeError: FirstOrderHold(): incompatible function arguments. The following argument types are supported:
    1. (arg0: numpy.ndarray[numpy.float64[m, 1]], arg1: numpy.ndarray[numpy.float64[m, n], flags.f_contiguous]) -> pydrake.trajectories.PiecewisePolynomial

Invoked with: [0.1, 0.2, 0.3], array([[[0.1, 0.2, 0.3],
        [0. , 0. , 0. ]],

       [[0. , 0. , 0. ],
        [0.1, 0.2, 0.3]]])

In [57]:
A = np.asfortranarray(A)
print(A.flags['F_CONTIGUOUS'])

True


In [58]:
PolyA = PiecewisePolynomial.FirstOrderHold(t, A)

TypeError: FirstOrderHold(): incompatible function arguments. The following argument types are supported:
    1. (arg0: numpy.ndarray[numpy.float64[m, 1]], arg1: numpy.ndarray[numpy.float64[m, n], flags.f_contiguous]) -> pydrake.trajectories.PiecewisePolynomial

Invoked with: [0.1, 0.2, 0.3], array([[[0.1, 0.2, 0.3],
        [0. , 0. , 0. ]],

       [[0. , 0. , 0. ],
        [0.1, 0.2, 0.3]]])